## Construcción de modelos

In [1]:
import pandas as pd
import torch
from catboost import CatBoostRegressor
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, learning_curve
from sklearn.experimental import enable_iterative_imputer
import numpy as np
from sklearn.metrics import (mean_absolute_error, r2_score,
                             root_mean_squared_error,
                             mean_absolute_percentage_error, mean_squared_error)

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, randint

import sys

from torch.utils.data import TensorDataset, DataLoader
from xgboost import XGBRegressor

from src.models.used_car_quote_nn import UsedCarQuoteNN

sys.path.append('../src')
from feature_engineering.name_splitter import NameSplitter
from feature_engineering.owner_mapper import OwnerMapper
from feature_engineering.seat_rounder import SeatRounder
from feature_engineering.max_power_converter import MaxPowerConverter
from feature_engineering.engine_converter import EngineConverter
from feature_engineering.multiple_interative_imputer import MultipleIterativeImputer
from feature_engineering.multiple_one_hot_encoder import MultipleOneHotEncoder
from feature_engineering.torque_standardizer import TorqueStandardizer
from feature_engineering.mileage_converter import MileageConverter
from feature_engineering.model_dropper import ModelDropper
from models.pipeline import CarsPipeline

-------------------
#### Data Processing

-------------------

Apalancados en la ingeniería de features, creamos un conjunto de clases a utilizar en la definición de pipelines, que nos permitan reproducir y modificar con facilidad los pasos de preprocesamiento, previos al entrenamiento de un modelo: 

In [2]:
data = pd.read_csv('../datasets/Car details v3.csv')

data["selling_price_log"] = np.log1p(data["selling_price"])

X = data.drop(columns=['selling_price', 'selling_price_log'])
y = data['selling_price']
y_log = data['selling_price_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=42)

In [3]:
final_pipeline = CarsPipeline()

X_train_processed = final_pipeline.fit_transform_df(X_train)
X_test_processed = final_pipeline.transform_df(X_test)


/Users/christianpisani/Learning/Posgrado/venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [4]:
missing_values = X_train_processed.isnull().sum()

for column, missing in missing_values.items():
    print(f"Feature: {column} - Missing values: {missing}")

X_train_processed

Feature: year - Missing values: 0
Feature: km_driven - Missing values: 0
Feature: owner - Missing values: 0
Feature: seats - Missing values: 0
Feature: mileage_kmpl - Missing values: 0
Feature: engine_cc - Missing values: 0
Feature: max_power_bhp - Missing values: 0
Feature: torque_peak_power - Missing values: 0
Feature: torque_peak_speed - Missing values: 0
Feature: fuel_Diesel - Missing values: 0
Feature: fuel_LPG - Missing values: 0
Feature: fuel_Petrol - Missing values: 0
Feature: seller_type_Individual - Missing values: 0
Feature: seller_type_Trustmark Dealer - Missing values: 0
Feature: transmission_Manual - Missing values: 0
Feature: make_Audi - Missing values: 0
Feature: make_BMW - Missing values: 0
Feature: make_Chevrolet - Missing values: 0
Feature: make_Daewoo - Missing values: 0
Feature: make_Datsun - Missing values: 0
Feature: make_Fiat - Missing values: 0
Feature: make_Force - Missing values: 0
Feature: make_Ford - Missing values: 0
Feature: make_Honda - Missing values: 0

,year,km_driven,owner,seats,mileage_kmpl,engine_cc,max_power_bhp,torque_peak_power,torque_peak_speed,fuel_Diesel,...,make_Mercedes-Benz,make_Mitsubishi,make_Nissan,make_Opel,make_Renault,make_Skoda,make_Tata,make_Toyota,make_Volkswagen,make_Volvo
0,-1.931351,0.833638,2.150221,-0.429567,0.059370,-1.338354,-1.279396,-1.078842,-0.078969,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
1,1.045338,0.498194,-0.643569,1.690873,-1.549766,2.611044,2.376872,2.435688,-0.723981,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,3.960213,-0.155906,-0.093209
2,0.797281,-0.524909,-0.643569,-0.429567,0.033340,1.078863,2.754477,1.801623,-0.347724,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
3,-0.194949,-0.524909,-0.643569,-0.429567,-1.223206,0.681706,1.336747,-0.073397,0.996052,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,3.960213,-0.155906,-0.093209
4,-0.194949,-0.004972,0.753326,-0.429567,0.934929,-0.427110,-0.489391,0.080591,-1.153989,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5684,-1.187179,0.833638,-0.643569,1.690873,-1.750907,1.449812,0.822530,0.986397,-0.293973,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
5685,0.053108,0.162750,0.753326,-0.429567,0.934929,-0.427110,-0.489391,0.080591,-1.153989,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
5686,0.549223,-0.591998,-0.643569,-0.429567,-0.200931,-0.529927,-0.266079,-0.601482,0.996052,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
5687,1.293396,-0.726176,-0.643569,-0.429567,2.118116,-0.427110,-0.488821,0.080591,-1.153989,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209


In [5]:
X_test_processed

,year,km_driven,owner,seats,mileage_kmpl,engine_cc,max_power_bhp,torque_peak_power,torque_peak_speed,fuel_Diesel,...,make_Mercedes-Benz,make_Mitsubishi,make_Nissan,make_Opel,make_Renault,make_Skoda,make_Tata,make_Toyota,make_Volkswagen,make_Volvo
0,-2.427466,0.665916,2.150221,-0.429567,-1.573429,0.066817,0.252130,-0.476381,1.641065,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
1,0.053108,3.718068,-0.643569,1.690873,-1.289464,1.449812,1.338743,1.258139,-0.401475,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,3.152838,-0.252512,-0.155906,-0.093209
2,0.549223,-0.004972,-0.643569,-0.429567,0.887601,-0.427110,-0.491102,0.080591,-1.153989,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
3,-1.931351,0.833638,0.753326,-0.429567,-0.603215,0.074881,0.252130,-0.440837,1.856069,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
4,-0.194949,-0.021744,0.753326,-0.429567,0.816610,-0.427110,-0.489391,0.080591,-1.153989,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,0.301166,-0.591998,-0.643569,-0.429567,-0.200931,-0.529927,-0.266079,-0.601482,0.996052,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
2435,-0.691064,0.330472,2.150221,0.630653,0.049520,0.890481,0.160131,0.128971,-0.446382,0.912577,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,3.960213,-0.155906,-0.093209
2436,0.797281,-0.843581,-0.643569,2.751093,-1.772205,-0.326308,-0.318271,-0.707461,1.533562,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,-0.169016,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209
2437,1.045338,-0.172693,-0.643569,-0.429567,0.842640,-0.929101,-0.689031,-0.816158,1.264807,-1.095798,...,-0.082003,-0.041963,-0.100602,-0.013259,5.916615,-0.109989,-0.317175,-0.252512,-0.155906,-0.093209


A continuación crearemos diversos modelos para luego compararlos.

-------------------
#### Ridge

-------------------

In [ ]:
model = Ridge()

ridge = GridSearchCV(
    model,
    {"alpha": np.linspace(0, 20, 1000)},
    refit=True,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    scoring='neg_mean_absolute_error'
)

ridge.fit(X_train_processed, y_train_log)

y_pred_ridge_log = ridge.predict(X_test_processed)
y_pred_ridge = np.expm1(y_pred_ridge_log)

metrics_ridge = {
    "name": "Ridge",
    "MAE_training": mean_absolute_error(y_train_log, ridge.predict(X_train_processed)),
    "MAE": mean_absolute_error(y_test, y_pred_ridge),
    "RMSE": root_mean_squared_error(y_test, y_pred_ridge),
    "MAPE": mean_absolute_percentage_error(y_test, y_pred_ridge),
    "R2": r2_score(y_test, y_pred_ridge)
}

-------------------
#### Random Forest

-------------------

In [ ]:
model = RandomForestRegressor(random_state=42)

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': [10, 20, None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'bootstrap': [True, False]
}

random_forest = RandomizedSearchCV(
    estimator=model,
    n_iter=50,
    param_distributions=param_dist,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    scoring='neg_mean_squared_error'
)

random_forest.fit(X_train_processed, y_train_log)

y_pred_random_forest_log = random_forest.predict(X_test_processed)
y_pred_random_forest = np.expm1(y_pred_random_forest_log)

metrics_rf = {
    "name": "Random Forest",
    "MAE_training": mean_absolute_error(y_train_log, random_forest.predict(X_train_processed)),
    "MAE": mean_absolute_error(y_test, y_pred_random_forest),
    "RMSE": root_mean_squared_error(y_test, y_pred_random_forest),
    "MAPE": mean_absolute_percentage_error(y_test, y_pred_random_forest),
    "R2": r2_score(y_test, y_pred_random_forest)
}

-------------------
#### CatBoost

-------------------

In [ ]:
model = CatBoostRegressor(
    eval_metric='RMSE',
    random_seed=42,
    verbose=0
)

params = {
    'iterations': [500, 1000, 1500],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'bagging_temperature': [0, 0.5, 1]
}

catboost = RandomizedSearchCV(
    estimator=model,
    param_distributions=params,
    n_iter=50,
    scoring='neg_root_mean_squared_error',
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    verbose=1,
    n_jobs=-1,
    random_state=42
)

catboost.fit(X_train_processed, y_train_log)

y_pred_catboost_log = catboost.predict(X_test_processed)
y_pred_catboost = np.expm1(y_pred_catboost_log)

metrics_catboost = {
    "name": "CatBoost",
    "MAE_training": mean_absolute_error(y_train_log, catboost.predict(X_train_processed)),
    "MAE": mean_absolute_error(y_test, y_pred_catboost),
    "RMSE": root_mean_squared_error(y_test, y_pred_catboost),
    "MAPE": mean_absolute_percentage_error(y_test, y_pred_catboost),
    "R2": r2_score(y_test, y_pred_catboost)
}

-------------------
#### XGBoost

-------------------

In [ ]:
model = XGBRegressor(random_state=42)

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(1, 2)
}

xgb = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train_processed, y_train_log)

y_pred_xgb_log = xgb.predict(X_test_processed)
y_pred_xgb = np.expm1(y_pred_xgb_log)

metrics_xgb = {
    "name": "XGBoost",
    "MAE_training": mean_absolute_error(y_train_log, xgb.predict(X_train_processed)),
    "MAE": mean_absolute_error(y_test, y_pred_xgb),
    "RMSE": root_mean_squared_error(y_test, y_pred_xgb),
    "MAPE": mean_absolute_percentage_error(y_test, y_pred_xgb),
    "R2": r2_score(y_test, y_pred_xgb)
}

-------------------
#### Neural Network

-------------------

In [ ]:
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X, y_log, test_size=0.3, random_state=42)
X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(X_train_nn, y_train_nn, test_size=0.3, random_state=42)

X_train_processed_nn = final_pipeline.fit_transform_df(X_train_nn)
X_test_processed_nn = final_pipeline.transform_df(X_test_nn)
X_val_processed_nn = final_pipeline.transform_df(X_val_nn)

X_train_tensor = torch.tensor(X_train_processed_nn.to_numpy(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_processed_nn.to_numpy(), dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_processed_nn.to_numpy(), dtype=torch.float32)

y_train_tensor = torch.tensor(y_train_nn.to_numpy(), dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test_nn.to_numpy(), dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val_nn.to_numpy(), dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
input_dim = X_train_processed_nn.shape[1]
model = UsedCarQuoteNN(input_size=input_dim)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.HuberLoss()

model.fit(train_loader=train_loader, test_loader=val_loader, optimizer=optimizer, loss_fn=loss_fn, epochs=100)

In [ ]:
model.eval()

y_pred_log_list = []
y_test_log_list = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        y_pred_log_list.append(y_pred.numpy())
        y_test_log_list.append(y_batch.numpy())

In [ ]:
y_pred_log_np = np.vstack(y_pred_log_list)
y_test_log_np = np.vstack(y_test_log_list)

y_pred_np = np.expm1(y_pred_log_np)
y_test_np = np.expm1(y_test_log_np)

metrics_nn = {
    "name": "NN",
    "MAE_log": mean_absolute_error(y_test_log_np, y_pred_log_np),
    "MAE": mean_absolute_error(y_test_np, y_pred_np),
    "RMSE": root_mean_squared_error(y_test_np, y_pred_np),
    "MAPE": mean_absolute_percentage_error(y_test_np, y_pred_np),
    "R2": r2_score(y_test_np, y_pred_np)
}

-------------------
#### Models Comparison

-------------------

In [ ]:
metrics = pd.DataFrame([metrics_ridge, metrics_rf, metrics_catboost, metrics_xgb, metrics_nn],)
metrics.sort_values(by=['MAE'], ascending=True)

En principio Random Forest parecería ser el modelo con mejor rendimiento. No solo por el bajo MAE sino también por el alto R2, el cuál indica que ajusta y generaliza bastante.

Si bien todos los modelos rinden de forma similar, Ridge presenta el peor desempeño. Esto se puede explicar por su naturaleza lineal, como mencionamos anteriormente, aunque el dataset tenga dimensiones con alta correlación entre ellas también tiene interacciones complejas que creemos Ridge no está logrando capturar adecuadamente.

Creemos, entonces, qué es lógico que aquellos modelos basados en árboles tengan mejor rendimiento que los modelos lineales. El proceso de cotizar un auto no siempre es lineal y depende de muchos factores con relaciones intrincadas. 

In [ ]:
catboost_best = catboost.best_estimator_
xgboost_best = xgb.best_estimator_
rf_best = random_forest.best_estimator_

catboost_importances = catboost_best.get_feature_importance()
xgboost_importances = xgboost_best.feature_importances_
rf_importances = rf_best.feature_importances_

features = X_train_processed.columns
importance_df = pd.DataFrame({
    'feature': features,
    'catboost': catboost_importances,
    'xgboost': xgboost_importances,
    'random_forest': rf_importances
})

importance_df['catboost'] /= importance_df['catboost'].sum()
importance_df['xgboost'] /= importance_df['xgboost'].sum()
importance_df['random_forest'] /= importance_df['random_forest'].sum()

importance_df = importance_df.sort_values(by='xgboost', ascending=False)

importance_df.plot(x='feature', kind='bar', figsize=(25, 10),
                   title="Comparación de Importancia de Características entre Modelos")
plt.xticks(rotation=90)
plt.ylabel("Importancia Normalizada")
plt.show()


Este gráfico muestra para cada modelo, excepto ridge, las dimensiones mejor valuadas. 

Es interesante como CatBoost y Random Forest no solo coinciden en varias sino también priorizan mucho el año y la fuerza del motor, dejando otras propiedades con menos peso. 

XGBoost además de distribuir los pesos de cada dimensión de forma más equilibrada tiene preferencia por algunas que no fueron consideradas por los modelos anteriores como la transmisión o si la marca es "Lexus". La última tiene sentido ya que es una marca perteneciente a Toyota dedicada a producir autos de lujo ([articulo de wikipedia](https://es.wikipedia.org/wiki/Lexus] articulo)) por lo tanto debe influir mucho en el precio del vehiculo.

In [ ]:
models = {
    'Ridge': ridge,
    'Random Forest': random_forest,
    'XGBoost': xgb,
    'CatBoost': catboost
}

mse_comparison = pd.DataFrame(
    [(name, mean_squared_error(y_test, np.expm1(model.predict(X_test_processed)))) for name, model in models.items()],
    columns=['Model', 'MSE']
)

mse_comparison.sort_values(by='MSE', ascending=True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))

# Gráfico 1: CatBoost
plt.subplot(2, 2, 1)
plt.scatter(y_test, y_pred_catboost, alpha=0.5, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.title("CatBoost")
plt.xlabel("Precio Real")
plt.ylabel("Predicción")

# Gráfico 2: XGBoost
plt.subplot(2, 2, 2)
plt.scatter(y_test, y_pred_xgb, alpha=0.5, color='green')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.title("XGBoost")
plt.xlabel("Precio Real")

# Gráfico 3: Random Forest
plt.subplot(2, 2, 3)
plt.scatter(y_test, y_pred_random_forest, alpha=0.5, color='orange')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.title("Random Forest")
plt.xlabel("Precio Real")
plt.ylabel("Predicción")

# Gráfico 4: Ridge
plt.subplot(2, 2, 4)
plt.scatter(y_test, y_pred_ridge, alpha=0.5, color='purple')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.title("Ridge Regression")
plt.xlabel("Precio Real")

# Ajustar diseño para evitar superposición
plt.tight_layout()

# Mostrar gráficos
plt.show()


Los gráficos que vemos muestran que tan precisas son las predicciones de los modelos, la línea roja es una recta que identifica el lugar donde la predicción fue perfecta. 

Se puede ver a simple vista que todos los modelos (excepto ridge) tienden a subestimar el valor de los autos cuanto más alto es el precio de los mismos. Se puede ver como hay más puntos bajo la línea roja a medida que crece el precio real. 

Ridge en cambio tiene una tendencia a subestimar y sobreestimar por igual.

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(2, 2, 1)
residuals_catboost = y_test - y_pred_catboost
plt.scatter(y_pred_catboost, residuals_catboost, alpha=0.5, color='blue')
plt.axhline(y=0, color='red', linestyle='--')
plt.title("CatBoost Residuos")
plt.xlabel("Predicción")
plt.ylabel("Residuos")

plt.subplot(2, 2, 2)
residuals_xgboost = y_test - y_pred_xgb
plt.scatter(y_pred_xgb, residuals_xgboost, alpha=0.5, color='green')
plt.axhline(y=0, color='red', linestyle='--')
plt.title("XGBoost Residuos")
plt.xlabel("Predicción")

plt.subplot(2, 2, 3)
residuals_rf = y_test - y_pred_random_forest
plt.scatter(y_pred_random_forest, residuals_rf, alpha=0.5, color='orange')
plt.axhline(y=0, color='red', linestyle='--')
plt.title("Random Forest Residuos")
plt.xlabel("Predicción")

plt.subplot(2, 2, 4)
residuals_rf = y_test - y_pred_ridge
plt.scatter(y_pred_ridge, residuals_rf, alpha=0.5, color='orange')
plt.axhline(y=0, color='red', linestyle='--')
plt.title("Ridge Residuos")
plt.xlabel("Predicción")
plt.tight_layout()

plt.show()


Este gráfico muestra los residuos (diferencia entre los valores reales y los predichos) en función de las predicciones para los cuatro modelos. Así podemos observar qué tan bien los modelos están ajustando los datos en diferentes rangos de predicción.

- A todos los modelos les cuesta predecir a medida que los precios suben. Confirmando lo analizado en los gráficos anteriores.
- CatBoost y XGBoost tienen distribuciones similares, uniformes y cercanas a cero.
- Random Forest logra predecir muy bien valores bajos pero a medida que los valores crecen se dispersa mucho más que XGBoost y CatBoost
- Ridge tiene la peor distribución de residuos, lo que confirma que no es adecuado para este tipo de problema con relaciones complejas y no lineales.